<div
    style='background-image: url("images/earthquake.jpg"); padding: 0px;
    background-size: cover; border-radius: 10px; height: 350px;
    background-position: 50% 50%'>
    <div
        style="position: relative; top: 75%; margin: 20px; padding: 10px;
        background: rgba(255 , 255 , 255 , 0.8); width: 95%; height: 80px;
        border-radius: 10px">
        <div
            style="position: relative; top: 50%;
            transform: translatey(-50%)">
            <div
                style="font-size: large; font-weight: 900;
                color: rgba(0 , 0 , 0 , 0.9);
                line-height: 100%">
                Introduction to seismo-acoustic waves in the Earth’s spheres
            </div>
            <div
                style="font-size: large; padding-top: 20px;
                color: rgba(0 , 0 , 0 , 0.7)">
                <p>Interpreting transient signals by <em>Shahar Shani-Kadmiel</em>, CEG, TU Delft
            </div>
        </div>
    </div>
</div>

# Interpreting transient signals
---
## Introduction
In this session we will discuss how array processing techniques can aid in the interpretation of transient signals. We will first go through the process of selecting an event and finding arrays that might have recorded it. We will then download array metadata and raw waveforms and try to get a feeling for what kind of signals we are able to see in the waveforms at different frequency bands.

Next we will use our array processing knowledge from the previous session to retrieve the wavefront parameters which will facilitate further analysis of the event.

## Retrieving data
There are several ways to retrieve data from various data services. We will look at the manual way, which uses a GUI-like interface in a web browser and the automatic way, which uses python code to directly interface with the online data services.

### IRIS GUI

[Wilber 3](https://ds.iris.edu/wilber3/find_event) is an Incorporated Research Institutions for Seismology (IRIS) data service that lets you select an earthquake and then select stations that have recorded that earthquake. It is possible to plot the waveforms prior to downloading the data.

Head over to the [Wilber 3](https://ds.iris.edu/wilber3/find_event) page and select the October 30, 2016 <i>M</i><sub>w</sub> 6.6 earthquake in Central Italy.

![IRIS__Wilber_3__Select_Event.png](images/IRIS__Wilber_3__Select_Event.png)

Next, look for all stations that have a `??F` (infrasound) channel. You should see I26H[1-8] (amongst others), which is an infrasound array of the IMS situated in Germany.

![IRIS__Wilber_3__Select_Stations.png](images/IRIS__Wilber_3__Select_Stations.png)

## Exercise

Go ahead and request some data... after you have downloaded the data, use ``obspy`` to read and plot the data:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from obspy import read

# wite some code here
raw_stream = read('path_to_your_data')



You may have noted that using this interface, you are limited in terms of the amount of data you can request.

Let's look at what python can do for us.

### ObsPy FDSN client

ObsPy is an open-source project dedicated to provide a Python framework for processing seismological data. It provides parsers for common file formats, clients to access data centers and seismological signal processing routines which allow the manipulation of seismological time series.

We will look at the [FDSN client](https://docs.obspy.org/packages/obspy.clients.fdsn.html) of ObsPy for retrieving station metadata and waveform data.

#### Fetching [event metadata](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.client.Client.get_events.html#obspy-clients-fdsn-client-client-get-events)

In [ ]:
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
client = Client('IRIS')

origintime = UTCDateTime('2016-10-30T06:40:19')

events = client.get_events(origintime - 5, origintime + 5, minmagnitude=6)
print(events)

event = events[0]
source_lon = event.origins[0].longitude
source_lat = event.origins[0].latitude
source_depth = event.origins[0].depth * 1e-3
source_mag = event.magnitudes[0].mag
origintime = event.origins[0].time

#### Fetching [station metadata](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.client.Client.get_stations.html#obspy-clients-fdsn-client-client-get-stations)

In [ ]:
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
client = Client('BGR')

origintime = UTCDateTime('2016-10-30T06:40:19')

inv = client.get_stations(
    origintime, network='*', station='I26*', channel='??F', level='response')
print(inv)

inv.write('../../Data/IS26.xml', 'StationXML')

We can already have a look at the instrument response. Try changing to different stations (array elements) to see if the instrument response is the same.

#### Fetching [waveforms](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.client.Client.get_waveforms.html#obspy-clients-fdsn-client-client-get-waveforms)

In [ ]:
t0 = origintime - 5 * 60
t1 = origintime + 3500
raw_stream = client.get_waveforms(
    network='GR', station='I26*', location='*', channel='BDF',
    starttime=t0, endtime=t1).sort()
print(raw_stream)
raw_stream.plot(size=(600, 600))

raw_stream.write(
    '../../Data/{}.mseed'.format(origintime.strftime('%Y%m%d%H%M%S')),
    format='MSEED'
)

## Seismic phase picking

ObsPy can also be used to calculate theoretical arrival times for arbitrary seismic phases in a 1D spherically symmetric background model. Furthermore it can output ray paths for all phases and derive pierce points of rays with model discontinuities:

In [ ]:
from obspy.taup import TauPyModel, plot_travel_times
model = TauPyModel(model='ak135')

phases = model.get_ray_paths(source_depth_in_km=source_depth,
                             distance_in_degree=6,
                             phase_list=['P', 'S', 'PcP', 'ScS', 'PKiKP'])
print(phases)

## Working with the Array object
The Array class inherits the functionality and hierarchy of the [ObsPy Inventory class](https://docs.obspy.org/packages/autogen/obspy.core.inventory.inventory.Inventory.html#obspy-core-inventory-inventory-inventory). The Array class provides some additional attributes and methods:

Attributes:

- ``name`` : The name of the array.
- ``aperture`` : The largest inter-station distance.
- ``center`` : Longitude, latitude, and elevation of the center of the array
- ``elements`` : A list of elements which are part of the array.
- ``nelements`` : Number of array elements.

Methods:

- ``get_coordinates()`` : Coordinates of array elements.
- ``plot_array_geometry()`` : Plot array configuration/geometry in Cartesian coordinates.
- ``plot_array_response()`` : Calculate and plot the array transfer response functions.

An Array instance can be initiated with an Inventory object already in memory or from a StationXML file:

In [ ]:
from arraylib import Array
array = Array(inv)

The instrument response of the sensors of this specific array is mostly flat between 0.1 and 5 Hz which is more or less the frequency band we are interested in.

In [ ]:
fig = array.plot_response(0.01, station='I26H1', show=False)
ax1, ax2 = fig.axes
ax1.set_ylim(3e3, 8e3)
ax1.axvspan(0.1, 5, color='r', alpha=0.1)
ax2.axvspan(0.1, 5, color='r', alpha=0.1)
plt.tight_layout()

print(inv.get_response('GR.I26H1..BDF', origintime))

In the array response plot, change the frequencies to try and figure out what is the frequency range the array is designed to be sensitive for.

In [ ]:
# set the plot
fwidth = 7
fheight = 4.5
fig = plt.figure(figsize=(fwidth, fheight))
wspace = 0.12
hspace = 0.05
left = 0.1
right = 0.15
bottom = 0.15
ncols = 2
width = (1 - left - right - wspace) / ncols
aspect = fwidth / fheight
height = width * aspect
ax1 = fig.add_axes((left, bottom,
                    width, height))
ax2 = fig.add_axes((left + width + wspace, bottom,
                    width, height))
ax02 = fig.add_axes((left + width + wspace, bottom + height + hspace,
                     width, 0.2), sharex=ax2)
plt.setp(ax02.get_xticklabels(), visible=False)

# array configuration
array.plot_array_geometry(ax1, c='r')

# array response
cbx = fig.add_axes((left + 2 * width + wspace + 0.015, bottom,
                    0.015, height))



# change frequencies here:
resp = array.plot_array_response(f_min=0.3, f_max=3, f_steps=1,
                                 ax=ax2, ax_top=ax02, cb_ax=cbx)

Recall that the raw waveforms did not look like much. Use the [filter](https://docs.obspy.org/packages/autogen/obspy.core.trace.Trace.filter.html#obspy-core-trace-trace-filter) method to filter out the low frequency oscillations and the high frequency noise. Bare in mind the instrument and array response that we have just discussed. Also, use the [remove_respose](https://docs.obspy.org/packages/autogen/obspy.core.trace.Trace.remove_response.html#obspy-core-trace-trace-remove-response) method to deconvolve with the instrument response.

In [ ]:
stream = raw_stream.copy()
stream.detrend('demean')
stream.taper(type='cosine', max_percentage=0.05)

# filter the data here:
stream.filter()


stream.remove_response(inv)
stream.plot(size=(600, 600))

To get better control over plotting we will define our own plotting function:

In [ ]:
PHASE_COLORS = {'P': 'b',
                'S': 'r',
                'PcP': 'g',
                'ScS': 'orange'}

def plot_waveforms(stream, origintime=None, phases=None):
    fig, ax = plt.subplots(stream.count(), 1, sharex=True, sharey=True,
                           figsize=(7, 1 * stream.count()))
    
    origintime = origintime or 0
    fig.subplots_adjust(hspace=0)
    for i, tr in enumerate(stream):
        times = tr.times()
        
        pre_origintime = origintime - tr.stats.starttime
        times -= pre_origintime
        
        axi = ax[i]
        axi.plot(times, tr.data, 'k', lw=0.5)
        axi.text(0.99, 0.97, tr.id, ha='right', va='top',
                 transform=axi.transAxes)
        
        # plot phases
        try:
            plotted = []
            for phase in phases:
                if phase.name in plotted:
                    continue
                    
                try:
                    color = PHASE_COLORS[phase.name]
                except KeyError:
                    color = 'k'
                axi.vlines(phase.time, tr.data.min(), tr.data.max(),
                           colors=color, label=phase.name)
                plotted.append(phase.name)
        except TypeError:
            pass
        
    axi.set_xlim(times[0], times[-1])
    if phases:
        axi.legend(loc=9, bbox_to_anchor=(0.5, -0.45), ncol=5, frameon=False)
    
    try:
        axi.set_xlabel('Time since origin {}'.format(
            origintime.strftime('%FT%T')
        ))
    except AttributeError:
        axi.set_xlabel('Time since {}'.format(
            tr.stats.starttime.strftime('%FT%T')
        ))
        
    try:
        for step in tr.stats.processing:
            if 'remove_response' in step or 'remove_sensitivity' in step:
                axi.set_ylabel('Pressure, Pa')
            else:
                axi.set_ylabel('Counts')
    except AttributeError:
        axi.set_ylabel('Counts')

In [ ]:
%matplotlib inline
plot_waveforms(stream, origintime, phases)

## Beamforming
We can now start the beamforming process to retrieve the wavefront parameters for the various signals in our data.

### Grid design

Yesterday we used a square slowness grid that was evenly spaced in slowness space:

In [ ]:
%matplotlib inline
from grid import Grid
grid = Grid(app_vel_params=(200, 50))
grid.plot_pxpy()

Note that there are many points in the corners outside the minimum velocity we are interested in. This obviously adds a computational overhead in the grid search process during beamforming. We also have no control over the resolution in velocity space which is unfavorable. For many reasons it makes more sense to do the grid search over a polar slowness grid with a known resolution in apparent velocity and back-azimuth.

In [ ]:
grid = Grid(app_vel_params=(280, 450, 10), theta_params=(0, 360, 2))
grid.plot_pxpy()

This grid will resolve wavefront parameters in the acoustic range and will treat signals outside this range as noise. In this case, we are interested in retrieving wavefront parameters over a large range of apparent velocities so it also makes sense to have an evenly spaced grid for the acoustic range and a lower resolution log-spaced grid for the seismic range:

In [ ]:
grid = Grid(app_vel_params=(280, 6000, 10, 450, 3), theta_params=(0, 360, 2))
grid.plot_pxpy()

### Preparing the data

We have already detrended, tapered, filtered, and deconvolved the instrument response. We need to make sure that all traces begin and end in the same time and have the same number of samples, and add the x, y attributes to each trace, the coordinates of the corresponding array element.

In [ ]:
stream = raw_stream.copy()
stream.detrend('demean')
stream.taper(type='cosine', max_percentage=0.05)

# filter the data here
stream.filter()


stream.remove_response(inv)

In [ ]:
# trim to the same start and end time
stream.trim(t0, t1, pad=True, fill_value=0)

# assign the x, y attributes and validate smapling rate and number of samples
samp_rate = []
npts = []
for tr in stream:
    element = array.select(station=tr.stats.station)[0][0]
    tr.x, tr.y = element.x, element.y
    samp_rate.append(tr.stats.sampling_rate)
    npts.append(tr.stats.npts)
    
if np.any(np.array(samp_rate) - tr.stats.sampling_rate):
    raise RuntimeError("Sampling rate does not match. {}".format(samp_rate))
if np.any(np.array(npts) - tr.stats.npts):
    raise RuntimeError("Number of samples does not match. {}".format(npts))

In [ ]:
from timefisher import beamform, fratio2snr, plot_results

bestbeam, times, fratio_max, baz, app_vel, fgrid = beamform(
    stream, grid, wlen=30, version='python')

snr = fratio2snr(fratio_max, stream.count())

In [ ]:
%matplotlib inline
from obspy.geodetics import gps2dist_azimuth

array_lon, array_lat, array_elev = array.center

distance, true_az, true_baz = gps2dist_azimuth(
    source_lat, source_lon, array_lat, array_lon
)
distance *= 1e-3

fig, ax, cb1, cb2 = plot_results(
    bestbeam, times, fratio_max, baz, app_vel, snr,
    stream, origintime, distance
)

# set the y-limits of the spectrogram frame
ax[0].set_ylim(0.3, 5)
ax[3].set_yscale('log')

In [ ]:
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
def plot_fgrid(times, bin, fgrid, origintime=None, cmap='inferno_r', ax=None,
               title=None):
    dt = times[1] - times[0]
    
    try:
        plt.sca(ax)
    except ValueError:
        ax = plt.gca()
    
    im = ax.tripcolor(grid.px, grid.py, fgrid[bin],
                 cmap=cmap, shading='gouraud')
    
    ax.set_aspect(1)
    divider = make_axes_locatable(ax)
    cbx = divider.append_axes('right', 0.15, 0.15)
    plt.colorbar(im, cax=cbx, label='Fisher ratio')
    
    ax.set_xlabel('px, s/m')
    ax.set_ylabel('py, s/m')
    if title is None:
        ax.set_title(
            ('Fisher ratio grid for time bin centered at:\n'
             f'{times[bin]:.2f} seconds')
        )
    
t_offset = bestbeam.stats.starttime - origintime
detection_times = times + t_offset
    
def update_plot(bin):
    plot_fgrid(detection_times, bin, fgrid)


    
from ipywidgets import interact, fixed, IntSlider
interact(
    update_plot,
    bin=IntSlider(
        value=0,
        min=0,
        max=times.size,
        step=5,
        continuous_update=False))

In [ ]:
%%javascript  # to have equation numbering
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

## Backprojections

Let's assume that each detection in the above array processing results is a signal that has propagated part of the way in the solid earth and the rest of the way in the atmosphere. As each detection point has a travel-time and a back-azimuth associated with it, we can map it to a geographical location relative to the array.

![Seismo-acoustic coupling cartoon](images/coupling_cartoon.png)

This is done by minimizing the misfit both in time and in back-azimuth in a grid search algorithm:

A travel-time matrix, $T_{ij}$, and a back-azimuth matrix, $BAZ_{ij}$, are constructed over the geographical extent of interest at $dh$ degrees grid spacing. Every grid point $(i, j)$ is treated as a potential point at which seismic waves may couple to infrasound. $T_{ij}$ is therefore the sum of the seismic travel-time, $T_{S,{ij}}$, from the hypocenter to each grid point:

$$
\begin{equation}
T_{S,{ij}} = (h^2 + Rs_{ij}^2)^{0.5}/c_s ,
\end{equation}
$$

and $T_{I,{ij}}$, the infrasonic travel-time from that grid point to the array:

\begin{equation}
T_{I,{ij}} = R_{I,{ij}}/c_I ,
\end{equation}

where $h$ is the hypocentral depth of the event, $R_{S,{ij}}$ and $R_{I,{ij}}$ are distance along a great circle of the seismic and infrasonic paths, respectively and $c_s$ and $c_I$ are seismic and infrasonic propagation velocities. Back-azimuth to each point is:


\begin{equation}
\label{eq:3}
BAZ_{ij} = \arctan \left( \frac{(\sin(\phi_{ij} - \phi_a)}
    {\cos(\lambda_a) \tan(\lambda_{ij}) -
     \sin(\lambda_a) \cos(\phi_{ij} - \phi_a)} \right) ,
\end{equation}


$\lambda_{ij}$, $\phi_{ij}$ are the latitude and longitude of a given point $(i, j)$, and $\lambda_a$, $\phi_a$ are the central coordinates of the array.

For each back-azimuth detection point, $BAZ_d$ at time $T_d$, we evaluate the joint misfit $M$ to be


\begin{equation}
M = M_{BAZ,{ij}} \cdot M_{T,{ij}},
\end{equation}


where $M_{BAZ,{ij}} = | BAZ_{ij} - BAZ_d |$ and $M_{T,{ij}} = | T_{ij} - T_d |$ are the back-azimuth and travel-time residual matrices, respectively. The grid point ($i, j$) which corresponds to $\min(M)$ is then most likely the point at which an infrasound detection with back-azimuth, $BAZ_d$ at time $T_d$, originated from.

In [ ]:
from backproject import backproject
backproject??

In [ ]:
# `times` aligned with the bestbeem which starts before origintime
# don't forget to subtract the pre-origintime

pre_origintime = origintime - bestbeam.stats.starttime
times_corrected = times - pre_origintime

# filter detections
detections = (
    (snr > 0.5) * 
    (times_corrected > 0) * 
    
    # similarly, filter out the detections from the north-west
)

T_d = times_corrected[detections]
BAZ_d = baz[detections]

# set the grid search extent (w, e, s, n)
grid_extent = (7, 18, 40, 50)

lons, lats = backproject(
    (source_lon, source_lat, source_depth), array.center,
    T_d, BAZ_d, extent=grid_extent, c_i=0.27
)

color_code = snr[detections]
sort_idx = color_code.argsort()
plt.close('all')
plt.scatter(lons[sort_idx], lats[sort_idx], 10, c=color_code[sort_idx],
            cmap='inferno_r')
plt.colorbar()

In [ ]:
import gdal
def read_GeoTIFF(filename):
    """
    Get data from GeoTIFF file.
    
    Parameters
    ----------
    filename : str
        Path (relative or absolute) to a GeoTIFF file.
        
    Returns
    -------
    z : :class:`~numpy.ndarray`
        Array (2d) of raster band data.
        
    (w, e, s, n) : tuple
        Extent of the data.
    """
    
    src_ds = gdal.Open(filename)
    band = src_ds.GetRasterBand(1)
    
    # elevation data
    z = band.ReadAsArray()
    
    # geo extent
    geotransform = src_ds.GetGeoTransform()
    w = geotransform[0]
    n = geotransform[3]
    dx = geotransform[1]
    dy = geotransform[5]
    
    nx = z.shape[1]
    ny = z.shape[0]
    e = w + nx * dx
    s = n + ny * dy
        
    return z, (w, e, s, n)


import shapefile
def read_shapefile(filename):
    """
    Get shapes from ESRI shapefile.
    
    Parameters
    ----------
    filename : str
        Path (relative or absolute) to a .shp file.
        
    Returns
    -------
    shapes : list
        List of all shapes in shapefile.
    """
    
    sf = shapefile.Reader(filename)
    return sf.shapes()

In [ ]:
# read shaded relief data
z, extent = read_GeoTIFF('../../Data/GEBCO_2014.hillshade.tiff')
fig, ax = plt.subplots(figsize=(6, 3))

# plot relief
im = ax.imshow(z, 'Greys_r', extent=extent, aspect='auto',
               interpolation='bilinear', vmin=50, vmax=220)

# read coastline data
shapes = read_shapefile('../../Data/ne_10m_coastline/ne_10m_coastline.shp')

# plot coastlines
for shape in shapes:
    x, y = np.array(shape.points).T
    ax.plot(x, y, 'k', lw=0.5, zorder=1)

In [ ]:
from matplotlib.patheffects import withStroke

fig, ax = plt.subplots(figsize=(6, 5))
ax.axis(grid_extent)

# plot relief
im = ax.imshow(z, 'Greys_r', extent=extent, aspect='auto',
               interpolation='bilinear', vmin=0, vmax=200, zorder=0)

# plot coastlines
for shape in shapes:
    x, y = np.array(shape.points).T
    ax.plot(x, y, 'k', lw=0.5,
            path_effects=[withStroke(linewidth=3, foreground="w")], zorder=2)

# plot source and receiver
ax.scatter(source_lon, source_lat, s=150, c='none', marker='*', edgecolor='k',
           path_effects=[withStroke(linewidth=3, foreground="w")], zorder=10)
ax.scatter(array_lon, array_lat, s=100, c='none', marker='^', edgecolor='k',
           path_effects=[withStroke(linewidth=3, foreground="w")], zorder=10)

# plot locations of seismo-acoustic coupling
sp = ax.scatter(lons[sort_idx], lats[sort_idx], 20,
                c=color_code[sort_idx], vmax=0.85 * color_code.max(),
                cmap='inferno_r', zorder=5)
plt.colorbar(sp, extend='both', label='SNR of detection', aspect=30)

### More refined backprojections

As we are interested in detections in the 90° -> 270° back azimuth range, we can limit our beamforming to that range instead of filtering the results in the post-processing stage:

In [ ]:
%matplotlib inline
grid = Grid(app_vel_params=(280, 6000, 5, 450, 3), theta_params=(90, 270, 1))
grid.plot_pxpy()

We might also want to tweak the bandpass filter parameters we used:

In [ ]:
stream = raw_stream.copy()
stream.detrend('demean')
stream.taper(type='cosine', max_percentage=0.05)
stream.filter('bandpass', freqmin=0.4, freqmax=3, corners=4, zerophase=True)
stream.remove_response(inv)

In [ ]:
# trim to the same start and end time
stream.trim(t0, t1, pad=True, fill_value=0)

# assign the x, y attributes and validate smapling rate and number of samples
samp_rate = []
npts = []
for tr in stream:
    element = array.select(station=tr.stats.station)[0][0]
    tr.x, tr.y = element.x, element.y
    samp_rate.append(tr.stats.sampling_rate)
    npts.append(tr.stats.npts)
    
if np.any(np.array(samp_rate) - tr.stats.sampling_rate):
    raise RuntimeError("Sampling rate does not match. {}".format(samp_rate))
if np.any(np.array(npts) - tr.stats.npts):
    raise RuntimeError("Number of samples does not match. {}".format(npts))

From this point on we will use a more optimized version of our beamforming algorithm that uses multi-threading as well to speed up processing. Add the ``version=numba`` keyword argument to the ``beamform()`` call.

In [ ]:
from timefisher import beamform, fratio2snr, plot_results

bestbeam, times, fratio_max, baz, app_vel, fgrid = beamform(
    stream, grid, wlen=15, overlap=0.9, version='numba')

snr = fratio2snr(fratio_max, stream.count())

In [ ]:
%matplotlib inline
from obspy.geodetics import gps2dist_azimuth

array_lon, array_lat, array_elev = array.center

distance, true_az, true_baz = gps2dist_azimuth(
    source_lat, source_lon, array_lat, array_lon
)
distance *= 1e-3

fig, ax, cb1, cb2 = plot_results(
    bestbeam, times, fratio_max, baz, app_vel, snr,
    stream, origintime, distance,
    vmin=0.5
)

ax[0].set_ylim(0.4, 5)
ax[2].set_ylim(90, 270)
ax[3].set_yscale('log')

In [ ]:
np.gradient(times)

In [ ]:
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
def plot_fgrid(times, bin, fgrid, origintime=None, cmap='inferno_r', ax=None,
               title=None):
    dt = times[1] - times[0]
    
    try:
        plt.sca(ax)
    except ValueError:
        ax = plt.gca()
    
    im = ax.tripcolor(grid.px, grid.py, fgrid[bin],
                 cmap=cmap, shading='gouraud')
    
    ax.set_aspect(1)
    divider = make_axes_locatable(ax)
    cbx = divider.append_axes('right', 0.15, 0.15)
    plt.colorbar(im, cax=cbx, label='Fisher ratio')
    
    ax.set_xlabel('px, s/m')
    ax.set_ylabel('py, s/m')
    if title is None:
        ax.set_title(
            ('Fisher ratio grid for time bin centered at:\n'
             f'{times[bin]:.2f} seconds')
        )
    
t_offset = bestbeam.stats.starttime - origintime
detection_times = times + t_offset
    
def update_plot(bin):
    plot_fgrid(detection_times, bin, fgrid)


    
from ipywidgets import interact, fixed, IntSlider
interact(
    update_plot,
    bin=IntSlider(
        value=250,
        min=0,
        max=times.size,
        step=5,
        continuous_update=False))

In [ ]:
# `times` aligned with the bestbeem which starts before origintime
# don't forget to subtract the pre-origintime

pre_origintime = origintime - bestbeam.stats.starttime
times_corrected = times - pre_origintime

# filter detections
detections = (
    (snr > 0.5) * 
    (times_corrected > 0)
)

T_d = times_corrected[detections]
BAZ_d = baz[detections]

# set the grid search extent (w, e, s, n)
grid_extent = (7, 18, 40, 50)

lons, lats = backproject(
    (source_lon, source_lat, source_depth), array.center,
    T_d, BAZ_d, extent=grid_extent, c_i=0.27
)

color_code = snr[detections]
sort_idx = color_code.argsort()


fig, ax = plt.subplots(figsize=(7.5, 5))
ax.axis(grid_extent)

# plot relief
im = ax.imshow(z, 'Greys_r', extent=extent, aspect='auto',
               interpolation='bilinear', vmin=0, vmax=200, zorder=0)

# plot coastlines
for shape in shapes:
    x, y = np.array(shape.points).T
    ax.plot(x, y, 'k', lw=0.5,
            path_effects=[withStroke(linewidth=3, foreground="w")], zorder=2)

# plot source and receiver
ax.scatter(source_lon, source_lat, s=150, c='none', marker='*', edgecolor='k',
           path_effects=[withStroke(linewidth=3, foreground="w")], zorder=10)
ax.scatter(array_lon, array_lat, s=100, c='none', marker='^', edgecolor='k',
           path_effects=[withStroke(linewidth=3, foreground="w")], zorder=10)

# make a histogram of the data 
w, e, s, n = grid_extent
H, Xe, Ye = np.histogram2d(lons, lats, bins=20, range=((w, e), (s, n)))
H = H.T

# plot histogram
cmap = plt.get_cmap('bone_r')
cmap.set_under('w')
cmap.set_over('k')
heatmap = ax.imshow(H, cmap, interpolation='gaussian', origin='lower',
                    aspect='auto', alpha=0.7, vmin=1, vmax=30,
                    extent=[Xe[0], Xe[-1], Ye[0], Ye[-1]], zorder=1)
cb = plt.colorbar(heatmap, extend='both', label='Number of detections',
                  aspect=30)
cb.solids.set_edgecolor("face")

# plot locations of seismo-acoustic coupling
sp = ax.scatter(lons[sort_idx], lats[sort_idx], 3,
                c=color_code[sort_idx], vmax=0.85 * color_code.max(),
                cmap='inferno_r', zorder=5)
plt.colorbar(sp, extend='both', label='SNR of detection', aspect=30)


This approach does not account for atmospheric effects such as velocity of the wind components and temperature variations that affect sound speeds and may result in inaccurate locations. The longer the propagation range, the more this approach is prone to error. The manifestation of such errors is visible as the epicentral infrasound detections are offset to the east relative to the epicenter. This is due to stratospheric cross-winds along the propagation path which leads to a discrepancy in back-azimuth.

# Exercise:

Now that you have a workflow for array processing and backprojection of transient seismo-acoustic signals, pick an earthquake from the list below and provide an analysis of the event.

Earthquakes:

1. Mw 9.1 2011-03-11 05:46:23
1. Mw 7.8 2012-10-28 03:04:07
1. Mw 7.5 2013-01-05 08:58:19
1. Mw 6.0 2016-08-24 01:36:32
1. Mw 5.9 2016-10-26 19:18:05
1. Mw 6.3 2017-09-03 03:30:01
1. Mw 7.9 2018-01-23 09:31:42
1. Mw 6.3 2018-08-12 14:58:54
1. Mw 7.0 2018-11-30 17:29:29

Consult us if you don't find data.